In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install pgmpy

Source: https://github.com/pgmpy/

Hoy en día, está bien establecido que el calentamiento global está causado en gran medida por los gases de efecto invernadero, que de hecho son responsables de atrapar el calor en la atmósfera. Los 3 gases más comunes son el dióxido de carbono (CO2), el metano (CH4) y el óxido nitroso (N2O). https://www.wri.org/insights/4-charts-explain-greenhouse-gas-emissions-countries-and-sectors

Hay varias fuentes de gases de efecto invernadero, como transporte, industria, comercial y residencial. En este notebook se aborda el problema de manera general, considerando el impacto del consumo energético en las emisiones de gases de efecto invernadero. De hecho, la energía está estrictamente relacionada con casi todos los factores de origen. En particular, enfrentaremos a la modelización de las relaciones causales entre el consumo de energía y los gases de efecto invernadero en Italia utilizando una red bayesiana. El objetivo es aprender un modelo que pueda proporcionar resultados probabilísticos dada alguna evidencia de entrada. Las relaciones causales y sus probabilidades relativas se estimarán analizando los factores de crecimiento anual de varios indicadores de conjuntos de datos de fuente abierta del Grupo del Banco Mundial (GBM). La opción de analizar el crecimiento anual tiene como objetivo captar cómo la variación de un indicador puede afectar a los demás.


### Definición de la red

En general, podemos esperar que el aumento del consumo de combustibles fósiles determine el crecimiento de la difusión de gases de efecto invernadero, mientras que un uso más amplio de energías renovables conduce a una reducción de las emisiones de gases de efecto invernadero. Se sugiere en algunos estudios que los factores de la tasa de crecimiento sobre la población, la urbanización y el producto interno bruto (PIB) pueden influir en el uso general de energía en una nación. Dados estos supuestos, la red bayesiana se define de la siguiente manera:


<code style="background-color:white;">
Pop  = <a href="https://data.worldbank.org/indicator/SP.POP.GROW?locations=IT">Population growth (annual %)</a>
Urb  = <a href="https://data.worldbank.org/indicator/SP.URB.GROW?locations=IT">Urban population growth (annual %)</a>
GDP  = <a href="https://data.worldbank.org/indicator/NY.GDP.PCAP.KD.ZG?locations=IT">GDP per capita growth (annual %)</a>
EC   = <a href="https://data.worldbank.org/indicator/EG.USE.PCAP.KG.OE?locations=IT">Energy use (kg of oil equivalent per capita)</a> - [annual growth %]
FFEC = <a href="https://data.worldbank.org/indicator/EG.USE.COMM.FO.ZS?locations=IT">Fossil fuel energy consumption (% of total)</a> - [annual growth %]
REC  = <a href="https://data.worldbank.org/indicator/EG.FEC.RNEW.ZS?locations=IT">Renewable energy consumption (% of total final energy consumption)</a> - [annual growth %]
EI   = <a href="https://data.worldbank.org/indicator/EG.IMP.CONS.ZS?locations=IT">Energy imports, net (% of energy use)</a> - [annual growth %]
CO2  = <a href="https://data.worldbank.org/indicator/EN.ATM.CO2E.PC?locations=IT">CO2 emissions (metric tons per capita)</a> - [annual growth %]
CH4  = <a href="https://data.worldbank.org/indicator/EN.ATM.METH.EG.KT.CE?locations=IT">Methane emissions in energy sector (thousand metric tons of CO2 equivalent)</a> - [annual growth %]
N2O  = <a href="https://data.worldbank.org/indicator/EN.ATM.NOXE.EG.KT.CE?locations=IT">Nitrous oxide emissions in energy sector (thousand metric tons of CO2 equivalent)</a> - [annual growth %]
</code>
    
    

In [ ]:
from pgmpy.models import BayesianModel

model = BayesianModel([('Pop', 'EC'),   ('Urb', 'EC'),   ('GDP', 'EC'),
                       ('EC', 'FFEC'),  ('EC', 'REC'),   ('EC', 'EI'),
                       ('REC', 'CO2'),  ('REC', 'CH4'),  ('REC', 'N2O'),
                       ('FFEC', 'CO2'), ('FFEC', 'CH4'), ('FFEC', 'N2O')])

### Datasets
Los **datos** para calcular las tablas de probabilidad condicional (CPT) de la red se encuentra en el csv compartido. Algunos datos se dan allí en valores absolutos y no en términos de **crecimiento anual**, por lo que es necesario transformarlos correctamente (las etiquetas de los datos transformados están marcadas con la cadena final "*- [annual growth %]*").

Una buena metodología para organizar **datos históricos para inferir CPT** es **agruparlos** **por año**, de modo que los valores históricos de la mismo año se tratan como una única tupla y, por lo tanto, constituyen una entrada completa del conjunto de datos. **La discretización de datos** también es necesaria, [ya que pgmpy no admite el aprendizaje de variables continuas] (https://github.com/pgmpy/pgmpy/issues/1084).


#### Raw data

In [ ]:
from pandas import read_csv, DataFrame
import numpy as np

def annual_growth(row, years):
    min_year = years["min"]
    max_year = years["max"]
    row["Indicator Name"] = row["Indicator Name"] + " - [annual growth %]"
    for year in range(max_year, min_year, -1):
        if not np.isnan(row[str(year)]) and not np.isnan(row[str(year - 1)]):
            row[str(year)] = 100 * (float(row[str(year)]) - float(row[str(year - 1)])) / abs(float(row[str(year - 1)]))
        else:
            row[str(year)] = np.nan     
    row[str(min_year)] = np.nan
    return row

years = {"min" : 1960, "max" : 2019}
df_raw = read_csv("/content/drive/My Drive/BME/BME_CLASES/DIRECTIVOS/BME PGM/EJERCICIOS_COMPLETOS_BAYESIANAS/italy-raw-data.csv")
df_raw_growth = DataFrame(data=[row if "growth" in row["Indicator Name"] else annual_growth(row, years) for index, row in df_raw.iterrows()])
df_raw_growth.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Italy,ITA,Population growth (annual %),SP.POP.GROW,1.993928,0.668383,0.676623,0.729553,0.822624,0.842109,0.777304,0.723778,0.631737,0.566059,0.528877,0.466453,0.567712,0.678188,0.654389,0.597247,0.498851,0.424722,0.356313,0.289147,0.206000,0.120054,0.074082,0.036295,0.022351,0.028900,0.005449,0.010205,0.048321,0.075009,0.083709,0.069231,0.067924,0.061136,0.020372,0.001589,0.028104,0.052907,0.028774,0.016821,0.045304,0.056168,0.148916,0.444507,0.647183,0.491389,0.300560,0.504934,0.662469,0.455613,0.307591,0.171978,0.269541,1.159251,0.917504,-0.096376,-0.169884,-0.149861,-0.190064,NaN
1,Italy,ITA,Urban population growth (annual %),SP.URB.GROW,2.836401,1.498807,1.506833,1.551287,1.636027,1.642485,1.568113,1.503610,1.403955,1.326087,1.278505,1.207417,1.013051,0.988290,0.962003,0.903919,0.804585,0.728011,0.657172,0.589108,0.503561,0.415237,0.147368,0.006388,-0.007565,-0.001023,-0.024485,-0.019735,0.018368,0.045050,0.053739,0.039254,0.121879,0.150993,0.110149,0.091284,0.117721,0.142444,0.118230,0.106198,0.134600,0.145384,0.297434,0.620956,0.823321,0.665742,0.474608,0.678682,0.835915,0.627293,0.480439,0.343066,0.619579,1.587835,1.341371,0.325701,0.246127,0.262999,0.228198,NaN
2,Italy,ITA,GDP per capita growth (annual %),NY.GDP.PCAP.KD.ZG,NaN,7.486419,5.487478,4.842052,1.955533,2.402046,5.164163,6.405678,5.873595,5.499180,4.713421,1.344280,3.103490,6.401806,4.812014,-2.673184,6.592320,2.125827,2.872936,5.653224,3.217170,0.723232,0.339225,1.132491,3.202783,2.768381,2.854367,3.181430,4.144042,3.310862,1.900440,1.468176,0.765808,-0.913402,2.130215,2.885202,1.238328,1.776351,1.781324,1.608635,3.739947,1.894124,0.104760,-0.305509,0.769318,0.323656,1.485157,0.975922,-1.615941,-5.711508,1.400915,0.534287,-3.242060,-2.972404,-0.917814,0.875477,1.451875,1.868715,0.966058,NaN
3,Italy,ITA,Energy use (kg of oil equivalent per capita) -...,EG.USE.PCAP.KG.OE,NaN,12.062200,13.064053,11.188621,9.110076,7.753922,8.706029,8.416381,9.468680,7.893911,10.571218,-3.803577,5.211975,6.088036,1.683184,-4.797379,7.501070,-2.644694,1.968338,4.114519,-0.923690,-2.264303,-2.620925,-0.121391,2.262151,1.379603,1.170196,4.140441,2.307209,4.194912,0.692845,2.391170,-0.692951,-0.584225,-1.283483,8.567489,-0.115062,1.367886,2.771110,1.522998,1.855858,0.279736,0.551071,4.321716,0.044299,1.411433,-1.209275,-0.825723,-1.968843,-7.049099,2.113919,-3.486796,-4.211107,-4.791839,-6.396212,2.786129,NaN,NaN,NaN,NaN
4,Italy,ITA,Fossil fuel energy consumption (% of total) - ...,EG.USE.COMM.FO.ZS,NaN,2.344018,1.933224,-0.167728,1.075163,-0.074481,0.552096,1.191740,0.880322,0.940778,0.013716,0.156887,0.291324,0.862886,-0.911534,-0.556895,0.817000,-0.953357,0.500252,0.375394,-0.121138,-0.404673,-0.816425,-0.072101,-0.979729,0.217271,-0.388764,2.105453,-0.512500,0.419555,0.153876,-0.580937,-0.338155,-0.335869,-0.197629,1.017196,-0.440740,-0.184842,-0.170454,-0.481676,-0.121162,-0.356122,-0.402127,-0.900475,0.210750,-0.668976,-0.679908,-1.129976,-1.520576,-2.292467,-0.262284,-0.193760,-2.679745,-2.721392,-1.723158,1.733165,NaN,NaN,NaN,NaN


#### Data cleaning

In [ ]:
nodes = ['Pop', 'Urb', 'GDP', 'EC', 'FFEC', 'REC', 'EI', 'CO2', 'CH4', 'N2O']
df_growth = df_raw_growth.transpose().iloc[4:]
df_growth.columns = nodes
df_growth.head(10)

,Pop,Urb,GDP,EC,FFEC,REC,EI,CO2,CH4,N2O
1960,1.99393,2.8364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961,0.668383,1.49881,7.48642,12.0622,2.34402,NaN,5.07261,13.8924,NaN,NaN
1962,0.676623,1.50683,5.48748,13.0641,1.93322,NaN,5.75395,17.5887,NaN,NaN
1963,0.729553,1.55129,4.84205,11.1886,-0.167728,NaN,2.51943,12.5116,NaN,NaN
1964,0.822624,1.63603,1.95553,9.11008,1.07516,NaN,0.631028,6.78298,NaN,NaN
1965,0.842109,1.64248,2.40205,7.75392,-0.0744814,NaN,2.33506,7.84845,NaN,NaN
1966,0.777304,1.56811,5.16416,8.70603,0.552096,NaN,1.51847,12.9005,NaN,NaN
1967,0.723778,1.50361,6.40568,8.41638,1.19174,NaN,1.938,9.41875,NaN,NaN
1968,0.631737,1.40395,5.87359,9.46868,0.880322,NaN,1.59627,6.42744,NaN,NaN
1969,0.566059,1.32609,5.49918,7.89391,0.940778,NaN,0.725698,8.22188,NaN,NaN


#### Data discretization

In [ ]:
TIERS_NUM = 3

def boundary_str(start, end, tier):
    return f'{tier}: {start:+0,.2f} to {end:+0,.2f}'

def relabel(v, boundaries):
    if v >= boundaries[0][0] and v <= boundaries[0][1]:
        return boundary_str(boundaries[0][0], boundaries[0][1], tier='A')
    elif v >= boundaries[1][0] and v <= boundaries[1][1]: 
        return boundary_str(boundaries[1][0], boundaries[1][1], tier='B')
    elif v >= boundaries[2][0] and v <= boundaries[2][1]:
        return boundary_str(boundaries[2][0], boundaries[2][1], tier='C')
    else:
        return np.nan
        
def get_boundaries(tiers):
    prev_tier = tiers[0]
    boundaries = [(prev_tier[0], prev_tier[prev_tier.shape[0] - 1])]
    for index, tier in enumerate(tiers):
        if index is not 0:
            boundaries.append((prev_tier[prev_tier.shape[0] - 1], tier[tier.shape[0] - 1]))
            prev_tier = tier
    return boundaries
    
new_columns = {}
for i, content in enumerate(df_growth.items()):  
    (label, series) = content
    values = np.sort(np.array([x for x in series.tolist() if not np.isnan(x)] , dtype=float))
    if values.shape[0] < TIERS_NUM:
        print(f'No ha suficientes datos {label}')
        break
    boundaries = get_boundaries(tiers=np.array_split(values, TIERS_NUM)) 
    new_columns[label] = [relabel(value, boundaries) for value in series.tolist()]
    
df = DataFrame(data=new_columns)
df.columns = nodes
df.index = range(years["min"], years["max"] + 1)
df.head(10)

,Pop,Urb,GDP,EC,FFEC,REC,EI,CO2,CH4,N2O
1960,C: +0.50 to +1.99,C: +0.82 to +2.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.79 to +13.06,C: +0.16 to +2.34,NaN,C: +0.78 to +5.75,C: +3.81 to +17.59,NaN,NaN
1962,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.79 to +13.06,C: +0.16 to +2.34,NaN,C: +0.78 to +5.75,C: +3.81 to +17.59,NaN,NaN
1963,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.79 to +13.06,B: -0.40 to +0.16,NaN,C: +0.78 to +5.75,C: +3.81 to +17.59,NaN,NaN
1964,C: +0.50 to +1.99,C: +0.82 to +2.84,B: +1.24 to +2.89,C: +2.79 to +13.06,C: +0.16 to +2.34,NaN,B: -0.38 to +0.78,C: +3.81 to +17.59,NaN,NaN
1965,C: +0.50 to +1.99,C: +0.82 to +2.84,B: +1.24 to +2.89,C: +2.79 to +13.06,B: -0.40 to +0.16,NaN,C: +0.78 to +5.75,C: +3.81 to +17.59,NaN,NaN
1966,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.79 to +13.06,C: +0.16 to +2.34,NaN,C: +0.78 to +5.75,C: +3.81 to +17.59,NaN,NaN
1967,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.79 to +13.06,C: +0.16 to +2.34,NaN,C: +0.78 to +5.75,C: +3.81 to +17.59,NaN,NaN
1968,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.79 to +13.06,C: +0.16 to +2.34,NaN,C: +0.78 to +5.75,C: +3.81 to +17.59,NaN,NaN
1969,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.79 to +13.06,C: +0.16 to +2.34,NaN,B: -0.38 to +0.78,C: +3.81 to +17.59,NaN,NaN


### Aprendizaje de los parámetros
En pgmpy es posible aprender el CPT de una red bayesiana dada usando un estimador bayesiano o un estimador de máxima verosimilitud (MLE). El primero explota una distribución previa conocida de datos, el segundo no hace ninguna suposición particular.

MLE puede sobreajustar los datos en el caso de conjuntos de datos pequeños, porque no puede haber suficientes observaciones y, por lo tanto, las frecuencias observadas pueden no ser representativas. Otro problema con MLE es el hecho de que los recuentos de estados se realizan de forma condicional para cada configuración de los padres y esto causa una inmensa fragmentación ya que los recuentos de estados caen aún más. En cambio, el estimador bayesiano no solo se basa en los datos de entrada para aprender los parámetros de la red, sino que también aprovecha un conocimiento previo, expresado a través de una distribución previa. De esta forma, el estimador no tiene una guía absoluta, sino un supuesto de partida razonable que permite contrarrestar la falta de datos.

Aunque el enfoque MLE parece plausible, puede ser demasiado simplista en muchos casos, mientras que el bayesiano es intrínsecamente más robusto.

Hay varias distribuciones previas disponibles en pgmpy, una elección sensata de prior es el Bayesian Dirichlet equivalent uniform prior (BDeu). En el proceso de aprendizaje, utilizando BDeu, se generan N muestras uniformes para cada variable para calcular los pseudo-counts (el valor predeterminado es N = 5), por lo tanto, las probabilidades estimadas en CPT son más conservadoras que las obtenidas a través de MLE (es decir, probabilidades cercanas a 1 o 0 se suavizan).


In [ ]:
from pgmpy.estimators import BayesianEstimator, MaximumLikelihoodEstimator
from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

model.cpds = []
model.fit(data=df, 
          estimator=BayesianEstimator,
          prior_type="BDeu",
          equivalent_sample_size=10,
          complete_samples_only=False)

print(f'Check model: {model.check_model()}\n')
for cpd in model.get_cpds():
    print(f'CPT de {cpd.variable}:')
    print(cpd, '\n')

Check model: True

CPT de Pop:
+------------------------+----------+
| Pop(A: -0.19 to +0.07) | 0.338164 |
+------------------------+----------+
| Pop(B: +0.07 to +0.50) | 0.338164 |
+------------------------+----------+
| Pop(C: +0.50 to +1.99) | 0.323671 |
+------------------------+----------+ 

CPT de EC:
+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+--------------

/usr/local/lib/python3.6/dist-packages/pgmpy/estimators/BayesianEstimator.py:155: UserWarning: pseudo count specified with bdeu prior. It will be ignored, use dirichlet prior for specifying pseudo_counts
  f"pseudo count specified with {prior_type} prior. It will be ignored, use dirichlet prior for specifying pseudo_counts"
/usr/local/lib/python3.6/dist-packages/pgmpy/estimators/BayesianEstimator.py:155: UserWarning: pseudo count specified with bdeu prior. It will be ignored, use dirichlet prior for specifying pseudo_counts
  f"pseudo count specified with {prior_type} prior. It will be ignored, use dirichlet prior for specifying pseudo_counts"
/usr/local/lib/python3.6/dist-packages/pgmpy/estimators/BayesianEstimator.py:155: UserWarning: pseudo count specified with bdeu prior. It will be ignored, use dirichlet prior for specifying pseudo_counts
  f"pseudo count specified with {prior_type} prior. It will be ignored, use dirichlet prior for specifying pseudo_counts"
/usr/local/lib/python3

### Análisis de la red Bayesiana

Entre todas las características, en pgmpy es posible investigar varias **propiedades de la red**. Por ejemplo, es posible verificar **independencia condicional** con respecto a alguna evidencia dada.


In [ ]:
from pgmpy.independencies.Independencies import IndependenceAssertion

def check_assertion(model, independent, from_variables, evidence):
    assertion = IndependenceAssertion(independent, from_variables, evidence)
    result = False
    for a in model.get_independencies().get_assertions():
        if frozenset(assertion.event1) == a.event1 and assertion.event2 <= a.event2 and frozenset(assertion.event3) == a.event3:
            result = True
            break
    print(f'{assertion}: {result}')

check_assertion(model, independent=['EI'], from_variables=['N2O'], evidence=['EC'])
check_assertion(model, independent=['EI'], from_variables=["FFEC", "REC", "GDP", "Pop", "Urb"], evidence=['EC'])
check_assertion(model, independent=['EC'], from_variables=["CH4"], evidence=['FFEC'])
check_assertion(model, independent=['EC'], from_variables=["CH4"], evidence=['FFEC', 'REC'])
check_assertion(model, independent=['FFEC'], from_variables=["REC"], evidence=['EC'])
check_assertion(model, independent=['FFEC'], from_variables=["REC"], evidence=['EC', 'CO2'])
check_assertion(model, independent=['CH4'], from_variables=["CO2"], evidence=['FFEC'])
check_assertion(model, independent=['CH4'], from_variables=["CO2"], evidence=['FFEC', 'REC'])

(EI _|_ N2O | EC): True
(EI _|_ FFEC, Urb, REC, GDP, Pop | EC): True
(EC _|_ CH4 | FFEC): False
(EC _|_ CH4 | REC, FFEC): True
(FFEC _|_ REC | EC): True
(FFEC _|_ REC | CO2, EC): False
(CH4 _|_ CO2 | FFEC): False
(CH4 _|_ CO2 | REC, FFEC): True


###  Inferencias


In [ ]:
from pgmpy.inference import VariableElimination
import time

def query_report(infer, variables, evidence=None, elimination_order="MinFill", show_progress=False, desc=""):
    if desc:
        print(desc)
    start_time = time.time()
    print(infer.query(variables=variables, 
                      evidence=evidence, 
                      elimination_order=elimination_order, 
                      show_progress=show_progress))
    print(f'--- Query ejecutada en {time.time() - start_time:0,.4f} segundos ---\n')
    
def get_ordering(infer, variables, evidence=None, elimination_order="MinFill", show_progress=False, desc=""):
    start_time = time.time()
    ordering = infer._get_elimination_order(variables=variables, 
                                        evidence=evidence, 
                                        elimination_order=elimination_order, 
                                        show_progress=show_progress)
    if desc:
        print(desc, ordering, sep='\n')
    return ordering

def padding(heuristic):
    return (heuristic + ":").ljust(16)

            
infer = VariableElimination(model)

var = ['CO2']
heuristic = "MinNeighbors"
ordering = get_ordering(infer, variables=var, elimination_order=heuristic,
                        desc="")
query_report(infer, variables=var, elimination_order=ordering, 
             desc=f'Probabilidad query de {var} sin evidencia:') 
query_report(infer, variables=var, elimination_order=list(reversed(ordering)), 
             desc=f'Probabilidad query de {var} sin evidencia:')

var = ['CO2']
ev = {'EC': 'A: -7.05 to -0.12'}
heuristic = "MinFill"
query_report(infer, variables=var, evidence=ev, elimination_order=heuristic, 
             desc=f'Probabilidad query de {var} con evidencia {ev} a traves {heuristic} heuristic:')
heuristic = "MinNeighbors"
query_report(infer, variables=var, evidence=ev, elimination_order=heuristic, 
             desc=f'Probabilidad query de {var} con evidencia {ev} a traves {heuristic} heuristic:') 

Probabilidad query de ['CO2'] sin evidencia:
+-------------------------+------------+
| CO2                     |   phi(CO2) |
+=========================+============+
| CO2(A: -10.20 to -0.79) |     0.3814 |
+-------------------------+------------+
| CO2(B: -0.79 to +3.81)  |     0.4011 |
+-------------------------+------------+
| CO2(C: +3.81 to +17.59) |     0.2175 |
+-------------------------+------------+
--- Query ejecutada en 0.0182 segundos ---

Probabilidad query de ['CO2'] sin evidencia:
+-------------------------+------------+
| CO2                     |   phi(CO2) |
+=========================+============+
| CO2(A: -10.20 to -0.79) |     0.3814 |
+-------------------------+------------+
| CO2(B: -0.79 to +3.81)  |     0.4011 |
+-------------------------+------------+
| CO2(C: +3.81 to +17.59) |     0.2175 |
+-------------------------+------------+
--- Query ejecutada en 0.0890 segundos ---

Probabilidad query de ['CO2'] con evidencia {'EC': 'A: -7.05 to -0.12'} a traves Min

Esta última parte de este documento intenta responder a algunas **preguntas clave** mediante consultas de probabilidad sobre el escenario de los gases de efecto invernadero:

- un crecimiento demográfico considerable conduce a un aumento del consumo de energía y a la difusión de CO<sub>2</sub>
- un aumento de la tasa de crecimiento de la urbanización y el PIB per cápita son factores importantes en la difusión del CO<sub> 2 </sub>
- la reducción del consumo de energía conduce a una reducción de la difusión de CO <sub> 2 </sub>
- un aumento de la difusión de CO <sub> 2 </sub> es evidencia de un aumento de la difusión de CH <sub> 4 </sub> y N <sub> 2 </sub> O
- una reducción de la difusión de CO <sub> 2 </sub> es evidencia de una reducción de la difusión de CH <sub> 4 </sub> y N <sub> 2 </sub> O
- La reducción del consumo de combustibles fósiles y el aumento del consumo de energía renovable tienen un impacto significativo en la difusión de gases de efecto invernadero.

In [ ]:
query_report(infer, variables=['EC'], evidence={'Pop': 'C: +0.50 to +1.99'},
             desc='Probabilidad de consumo de la energia con  crecimiento de la población por enciama de +0.50%:')
query_report(infer, variables=['CO2'], evidence={'Pop': 'C: +0.50 to +1.99'},
             desc='Probabilidad de crecimiento de CO2 con population growth over +0.50%:')
query_report(infer, variables=['CO2'], evidence={'Urb': 'C: +0.82 to +2.84'}, 
             desc='Probabilidad de crecimiento de CO2 icon urbanization growth over +0.82%:')
query_report(infer, variables=['CO2'], evidence={'GDP': 'C: +2.89 to +7.49'}, 
             desc='Probabilidad de crecimiento deCO2 con GDP per capita growth over +2.89%:')

Probabilidad de consumo de la energia con  crecimiento de la población por enciama de +0.50%:
+------------------------+-----------+
| EC                     |   phi(EC) |
+========================+===========+
| EC(A: -7.05 to -0.12)  |    0.3087 |
+------------------------+-----------+
| EC(B: -0.12 to +2.79)  |    0.3011 |
+------------------------+-----------+
| EC(C: +2.79 to +13.06) |    0.3902 |
+------------------------+-----------+
--- Query ejecutada en 0.0340 segundos ---

Probabilidad de crecimiento de CO2 con population growth over +0.50%:
+-------------------------+------------+
| CO2                     |   phi(CO2) |
+=========================+============+
| CO2(A: -10.20 to -0.79) |     0.3704 |
+-------------------------+------------+
| CO2(B: -0.79 to +3.81)  |     0.4008 |
+-------------------------+------------+
| CO2(C: +3.81 to +17.59) |     0.2288 |
+-------------------------+------------+
--- Query ejecutada en 0.0308 segundos ---

Probabilidad de crecimiento 

Podemos obtener los siguientes **resultados** de las consultas en la red bayesiana modelada:

- **Consumo de energía frente a población, urbanización, PIB**: el aumento de la difusión de gases es más claramente proporcional al crecimiento del consumo de energía que al crecimiento de la población, la urbanización o el PIB per cápita. Este resultado puede ser un síntoma de que algunos factores casuales importantes del crecimiento del consumo de energía que estan estrictamente correlacionados con la difusión de gases de efecto invernadero, como la industrialización, no están representados en la red.
- <b> N <sub> 2 </sub> O vs CH <sub> 4 </sub> </b>: Las tendencias de crecimiento de N <sub> 2 </sub> O casi se parecen a las del CO <sub> 2 </sub>, mientras que el crecimiento de CH <sub> 4 </sub> parece muy probable que disminuya independientemente de la evidencia. Es poco probable que esta sea la situación real, y la consulta de probabilidad de CH <sub> 4 </sub> sin evidencia dada lo demuestra: el nivel inferior 'A' es mucho más probable que los demás, y eso es una consecuencia de los datos escasez. Los otros 2 niveles de CH <sub> 4 </sub> están subrepresentados. Ese no es el caso de N <sub> 2 </sub> O, cuyos niveles están más equilibrados.
- **Combustible fósil vs Renovable**: finalmente y lo más importante, las probabilidades de salida de las últimas consultas sugieren que la reducción del consumo de energía de combustibles fósiles y el aumento del consumo de energía renovable son estrategias realmente efectivas para minimizar las emisiones de gases de efecto invernadero, de ahí que Es realmente importante utilizar energías sostenibles.
